<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/%EB%B9%85%EC%9D%91%EB%B3%B4_case1_2_AE_IF_LOF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:

data = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type1_normal.csv')#.to_numpy()
y = data['type']
y = y.map({'Normal': 0, 'Attack': 1}).to_numpy()
#X['messageID'] = data['messageID']
X = data.drop(columns =['type'])
#X = X.drop(columns =['isCan0','isService','serviceNODEID'])
X = X.drop(columns = ['time'])#,'data[0]','data[7]'])
print(X.shape)


(91042, 16)


In [42]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
input_dim = X_scaled.shape[1]
encoding_dim = 14

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Autoencoder 학습
autoencoder.fit(X_scaled, X_scaled, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

Epoch 1/50
2277/2277 [==============================] - 7s 3ms/step - loss: 0.6329 - val_loss: 0.6193
Epoch 2/50
2277/2277 [==============================] - 5s 2ms/step - loss: 0.5602 - val_loss: 0.5981
Epoch 3/50
2277/2277 [==============================] - 5s 2ms/step - loss: 0.5566 - val_loss: 0.5945
Epoch 4/50
2277/2277 [==============================] - 7s 3ms/step - loss: 0.5555 - val_loss: 0.5947
Epoch 5/50
2277/2277 [==============================] - 5s 2ms/step - loss: 0.5552 - val_loss: 0.5943
Epoch 6/50
2277/2277 [==============================] - 7s 3ms/step - loss: 0.5550 - val_loss: 0.5952
Epoch 7/50
2277/2277 [==============================] - 5s 2ms/step - loss: 0.5549 - val_loss: 0.5933
Epoch 8/50
2277/2277 [==============================] - 5s 2ms/step - loss: 0.5548 - val_loss: 0.5930
Epoch 9/50
2277/2277 [==============================] - 7s 3ms/step - loss: 0.5548 - val_loss: 0.5935
Epoch 10/50
2277/2277 [==============================] - 5s 2ms/step - loss: 0.554

In [43]:
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)
encoder_model = Model(inputs=input_layer, outputs=encoder)

# 데이터를 인코딩
X_encoded = encoder_model.predict(X_scaled)

2846/2846 [==============================] - 4s 1ms/step


In [44]:
#data.loc[data['type']=='Normal', 'type'] = 0
#data.loc[data['type']=='Attack', 'type'] = 1

fullDataX1 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type1.csv')#.to_numpy()

#X['messageID'] = data['messageID']
fX1 = fullDataX1.drop(columns =['type'])
#fX1 = fX1.drop(columns =['isCan0','isService','serviceNODEID'])
fX1 = fX1.drop(columns = ['time'])#,'data[0]','data[7]'])
print(fX1.shape)
fX1_scaled = scaler.fit_transform(fX1)
fX1_encoded = encoder_model.predict(fX1_scaled)

(207858, 16)
6496/6496 [==============================] - 10s 1ms/step


In [45]:
if_model = IsolationForest(contamination=0.1)  # contamination 값 조정 가능
if_model.fit(X_encoded)

# 이상치 예측
if_predictions = if_model.predict(fX1_encoded)

In [46]:
# Local Outlier Factor 모델 정의 및 학습
lof_model = LocalOutlierFactor(n_neighbors=20, contamination=0.1, novelty=True)  # n_neighbors 및 contamination 값 조정 가능
lof_model.fit(X_encoded)
lof_predictions = lof_model.predict(fX1_encoded)
# 이상치라면 -1, 정상치라면 1로 표시됨


In [47]:
print(np.unique(if_predictions, return_counts=True))

(array([-1,  1]), array([ 14350, 193508]))


In [48]:
print(np.unique(lof_predictions, return_counts=True))

(array([-1,  1]), array([206511,   1347]))


In [49]:
data2 = pd.read_csv('./drive/MyDrive/Colab Notebooks/team_prj/data/type2.csv')
y2 = data2['type']
y2 = y2.map({'Normal': 0, 'Attack': 1}).to_numpy()
#X2 = data2.drop(columns =['type'])
#X2 = X2.drop(columns =['isCan0'])
#X2 = X2.drop(columns = ['time'])

X2 = data2.drop(columns =['type'])
X2 = X2.drop(columns =['isCan0','isService','serviceNODEID'])
X2 = X2.drop(columns = ['time','data[0]','data[7]'])

